In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In this notebook we'll explore feature importance using SHAP values. SHAP values are the most mathematically consistent way for getting feature importances, and they work particulalry nicely with the tree-based models. Unfortunately, calculating SHAP values is an extremely resource intensive process. However, starting with XGBoost 1.3 it is possible to calcualte these values on GPUs, which speeds up the process by a factor of 20X - 50X compared to calculating the same on a CPU. Furthermore, it is also possible to calculate SHAP values for feature interactions. The GPU speedup for those is even more dramatic - it takes a few minutes, as opposed to days or even longer on a CPU.

In [ ]:
%matplotlib inline
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
import shap

# load JS visualization code to notebook
shap.initjs()
xgb.__version__

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
claim = train['claim'].values

In [ ]:
cat_features = columns[:19]
cat_features

In [ ]:
def label_encode(train_df, test_df, column):
    le = LabelEncoder()
    new_feature = "{}_le".format(column)
    le.fit(train_df[column].unique().tolist() + test_df[column].unique().tolist())
    train_df[new_feature] = le.transform(train_df[column])
    test_df[new_feature] = le.transform(test_df[column])
    return new_feature

In [ ]:
cat_cols = [col for col in columns if 'cat' in col]
cont_cols = [col for col in columns if 'cont' in col]

In [ ]:
le_cols = []
for feature in cat_cols:
    le_cols.append(label_encode(train, test, feature))

In [ ]:
columns = le_cols + cont_cols

In [ ]:
xgb_params=  {'learning_rate': 0.005,
              'objective': 'binary:logistic',
              'eval_metric': 'auc',
                'tree_method': 'gpu_hist',
                'predictor': 'gpu_predictor',
                'gpu_id': 0,
                'max_bin': 623,
                'max_depth': 10,
                'alpha': 0.5108154566815425,
                'gamma': 1.9276236172849432,
                'reg_lambda': 11.40999855634382,
                'colsample_bytree': 0.705851334291963,
                'subsample': 0.8386116751473301,
                'min_child_weight': 2.5517043283716605}

In [ ]:
test = xgb.DMatrix(test[columns])

In [ ]:
train_oof = np.zeros((train[columns].shape[0],))
test_preds = 0
train_oof_shap = np.zeros((train[columns].shape[0],train[columns].shape[1]+1))
test_preds_shap = 0
train_oof_shap.shape

In [ ]:
NUM_FOLDS = 10
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, claim))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        train_claim, val_claim = claim[train_ind], claim[val_ind]
        
        train_df = xgb.DMatrix(train_df, label=train_claim)
        val_df = xgb.DMatrix(val_df, label=val_claim)
        
        model =  xgb.train(xgb_params, train_df,7000)
        temp_oof = model.predict(val_df)
        temp_oof_shap = model.predict(val_df, pred_contribs=True)
        temp_test = model.predict(test)
        temp_test_shap = model.predict(test, pred_contribs=True)

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        train_oof_shap[val_ind, :] = temp_oof_shap
        test_preds_shap += temp_test_shap/NUM_FOLDS
        
        
        print(roc_auc_score(val_claim, temp_oof))
        
print(roc_auc_score(claim, train_oof))